# [TUTORIAL] Constrained Task Planning (Sweep)

This tutorial is written to instruct basic usage of the task & motion planning pipeline  
One Indy7 robot and several environment geometries will be added and floor-wiping task will be conducted.  
Here, the wiping task is defined as 1) contact with floor, 2) starting waypoint, 3) goal waypoint.  
Thus, any motion that satisfies the constraint will be generated; it may not look like real wiping.  

## set running directory to project source

In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, None,
                INDY_IP)]
              , connection_list=[False])

connection_list
[False]


## create scene builder

In [3]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None, base_link="base_link")
# s_builder.reset_reference_coord(ref_name="floor")

## get ghnd with detected robot config

In [4]:
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {"indy0": ((0,0,0), (0,0,0))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]


## add environment

In [5]:
from pkg.geometry.geometry import *

floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (0.3,0.5,0.01), (0.5,0,-0.005), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=False)
floor_col = gscene.create_safe(GEOTYPE.BOX, "floor_col", "base_link", (0.3,0.5,0.01), (0.5,0,-0.006), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=False, fixed=True, collision=True)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.2,0,0), 
                           rpy=(0,np.pi/2,), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
wp11 = gscene.create_safe(GEOTYPE.BOX, "wp11", "base_link", (0.1,0.1,0.01), (0.6,0.2,-0.005),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False)
wp12 = gscene.create_safe(GEOTYPE.BOX, "wp12", "base_link", (0.1,0.1,0.01), (0.6,0.1,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False)
wp13 = gscene.create_safe(GEOTYPE.BOX, "wp13", "base_link", (0.1,0.1,0.01), (0.6,0.0,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False)
wp14 = gscene.create_safe(GEOTYPE.BOX, "wp14", "base_link", (0.1,0.1,0.01), (0.6,-0.1,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False)
wp15 = gscene.create_safe(GEOTYPE.BOX, "wp15", "base_link", (0.1,0.1,0.01), (0.6,-0.2,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False)
wp21 = gscene.create_safe(GEOTYPE.BOX, "wp21", "base_link", (0.1,0.1,0.01), (0.5,-0.2,-0.005),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False)
wp22 = gscene.create_safe(GEOTYPE.BOX, "wp22", "base_link", (0.1,0.1,0.01), (0.5,-0.1,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False)
wp23 = gscene.create_safe(GEOTYPE.BOX, "wp23", "base_link", (0.1,0.1,0.01), (0.5,0.0,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False)
wp24 = gscene.create_safe(GEOTYPE.BOX, "wp24", "base_link", (0.1,0.1,0.01), (0.5,0.1,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False)
wp25 = gscene.create_safe(GEOTYPE.BOX, "wp25", "base_link", (0.1,0.1,0.01), (0.5,0.2,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False)
wp31 = gscene.create_safe(GEOTYPE.BOX, "wp31", "base_link", (0.1,0.1,0.01), (0.4,0.2,-0.005),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False)
wp32 = gscene.create_safe(GEOTYPE.BOX, "wp32", "base_link", (0.1,0.1,0.01), (0.4,0.1,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False)
wp33 = gscene.create_safe(GEOTYPE.BOX, "wp33", "base_link", (0.1,0.1,0.01), (0.4,0.0,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False)
wp34 = gscene.create_safe(GEOTYPE.BOX, "wp34", "base_link", (0.1,0.1,0.01), (0.4,-0.1,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False)
wp35 = gscene.create_safe(GEOTYPE.BOX, "wp35", "base_link", (0.1,0.1,0.01), (0.4,-0.2,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False)
track = gscene.create_safe(GEOTYPE.BOX, "track", "base_link", (0.1,0.5,0.01), (0.5,0,-0.005), rpy=(0,0,0), 
                           color=(1,0.7,0.7,0.5), display=True, fixed=True, collision=False)
goal = gscene.create_safe(GEOTYPE.BOX, "goal", "base_link", (0.1,0.1,0.01), (0.3,-0.4,-0.005), 
                          rpy=(0,0,0), color=(0.8,0.8,0.2,1), display=True, fixed=True, collision=False)

Please create a subscriber to the marker


In [6]:
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True, exclude_link=["panda1_link7"])

## add brush

In [7]:
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_body", link_name="base_link", dims=(0.1,0.07,0.02), 
                   center=(0.2,0.4,0.04), rpy=(0,0,0), color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False)
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_handle", link_name="base_link", dims=(0.1,0.03,0.05), center=(0,0,0.035), rpy=(0,0,0), 
                   color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False, parent="brush_body")
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face", link_name="base_link", dims=(0.1,0.06,0.03), center=(0,0,-0.025), rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,1), display=True, collision=False, fixed=False, parent="brush_body")

## add box

In [8]:
gscene.create_safe(gtype=GEOTYPE.BOX, name="obstacle", link_name="base_link", dims=(0.05,0.05,0.05), 
                   center=(0.5,0.0,0.025), rpy=(0,0,0), color=(0.7,0.3,0.3,1), display=True, collision=True, fixed=False)

## init planning scene

In [9]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

## Register binders

In [10]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool

In [11]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="indy0_tcp", 
                 dims=(0.01,)*3, center=(0,0,0.14), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

In [12]:
pscene.create_binder(bname="grip0", gname="grip0", rname="indy0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)
pscene.create_binder(bname="goal_bd", gname="goal", _type=PlacePlane, point=(0,0,0.005), rpy=(0,0,0))
pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepTool, point=(0,0,-0.015), rpy=(0,0,0))

## add objects

In [13]:
from pkg.planning.constraint.constraint_subject import CustomObject, Grasp2Point, PlacePoint, SweepPoint, SweepTask, BoxObject

In [14]:
brush_handle = gscene.NAME_DICT["brush_handle"]
brush_face = gscene.NAME_DICT["brush_face"]
brush = pscene.create_object(oname="brush", gname="brush_body", _type=CustomObject, 
                             action_points_dict = {"handle": Grasp2Point("handle", brush_handle, [0,0,0], [np.pi/2,0,0]),
                                                   "face": PlacePoint("face", brush_face, [0,0,-0.015], [0,0,0])})

In [15]:
obstacle = pscene.create_object(oname="obstacle", gname="obstacle", _type=BoxObject, hexahedral=True)

In [16]:
sweep = pscene.create_object(oname="sweep", gname="floor", _type=SweepTask, 
                             action_points_dict = {"wp11": SweepPoint("wp11", wp11, [0,0,0.005], [0,0,0]),
                                                   "wp12": SweepPoint("wp12", wp12, [0,0,0.005], [0,0,0]),
                                                   "wp13": SweepPoint("wp13", wp13, [0,0,0.005], [0,0,0]),
                                                   "wp14": SweepPoint("wp14", wp14, [0,0,0.005], [0,0,0]),
                                                   "wp15": SweepPoint("wp15", wp15, [0,0,0.005], [0,0,0]),
                                                   "wp21": SweepPoint("wp21", wp21, [0,0,0.005], [0,0,0]),
                                                   "wp22": SweepPoint("wp22", wp22, [0,0,0.005], [0,0,0]),
                                                   "wp23": SweepPoint("wp23", wp23, [0,0,0.005], [0,0,0]),
                                                   "wp24": SweepPoint("wp24", wp24, [0,0,0.005], [0,0,0]),
                                                   "wp25": SweepPoint("wp25", wp25, [0,0,0.005], [0,0,0]),
                                                   "wp31": SweepPoint("wp31", wp31, [0,0,0.005], [0,0,0]),
                                                   "wp32": SweepPoint("wp32", wp32, [0,0,0.005], [0,0,0]),
                                                   "wp33": SweepPoint("wp33", wp33, [0,0,0.005], [0,0,0]),
                                                   "wp34": SweepPoint("wp34", wp34, [0,0,0.005], [0,0,0]),
                                                   "wp35": SweepPoint("wp35", wp35, [0,0,0.005], [0,0,0])})

### planning pipeline

In [17]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)
ppline.set_motion(mplan)
ppline.set_sampler(tplan)

## motion filters

In [18]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker
TOOL_LINK = "indy0_tcp"
TOOL_LINK_BUNDLE = ["indy0_tcp", "indy0_link6"]

gcheck = GraspChecker(pscene, end_link_couple_dict= {TOOL_LINK: TOOL_LINK_BUNDLE, "base_link":["base_link"]})
rcheck = ReachChecker(pscene)
checkers_all = [gcheck, rcheck]
# lcheck = LatticedChecker(pscene, end_link_couple_dict= {TOOL_LINK: TOOL_LINK_BUNDLE, "base_link":["base_link"]})
# checkers_all.append(lcheck)

In [19]:
mplan.motion_filters = checkers_all

## ui

In [20]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


```
open web ui on <your ip>:8050
click geometry items / Handles / Binders to highlight geometry on RVIZ
other functions may be buggy.. please report
```

## Test plan

In [21]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy
gtimer = GlobalTimer.instance()
initial_state = pscene.update_state(crob.home_pose)

   Use a production WSGI server instead.
 * Debug mode: off


In [22]:
gtimer.reset()
# goal_nodes = [("goal",)]
goal_nodes = [("goal", "floor", 15)]
gtimer.tic("plan")
ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, timeout_loop=300, multiprocess=True)
gtimer.toc("plan")
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.sort_schedule(schedules)
snode_schedule = ppline.idxSchedule2SnodeScedule(schedules_sorted[0])

Use 20/20 agents
node: ('floor', 'floor', 0)->('grip0', 'floor', 0) = fail
node: ('floor', 'floor', 0)->('grip0', 'floor', 0) = fail
node: ('floor', 'floor', 0)->('floor', 'floor', 1) = fail
node: ('floor', 'floor', 0)->('floor', 'floor', 1) = fail
node: ('floor', 'floor', 0)->('floor', 'floor', 1) = fail
node: ('floor', 'floor', 0)->('floor', 'grip0', 0) = fail
node: ('floor', 'floor', 0)->('floor', 'grip0', 0) = fail
node: ('floor', 'floor', 0)->('floor', 'grip0', 0) = fail
node: ('floor', 'floor', 0)->('grip0', 'floor', 0) = fail
node: ('floor', 'floor', 0)->('floor', 'grip0', 0) = fail
node: ('floor', 'floor', 0)->('grip0', 'floor', 0) = fail
node: ('floor', 'floor', 0)->('floor', 'floor', 1) = fail
node: ('floor', 'floor', 0)->('floor', 'floor', 1) = fail
node: ('floor', 'floor', 0)->('floor', 'floor', 1) = fail
node: ('floor', 'floor', 0)->('grip0', 'floor', 0) = fail
node: ('floor', 'floor', 0)->('grip0', 'floor', 0) = fail
node: ('floor', 'floor', 0)->('grip0', 'floor', 0) = su

branching: 4->18 (0.73/300.0 s, steps/err: 22(66.8811798096 ms)/0.00158454100732)
node: ('goal', 'grip0', 0)->('goal', 'goal', 0) = fail
node: ('grip0', 'floor', 1)->('grip0', 'floor', 2) = fail
node: ('grip0', 'floor', 1)->('floor', 'floor', 1) = fail
node: ('grip0', 'floor', 0)->('grip0', 'floor', 1) = success
branching: 1->20 (0.8/300.0 s, steps/err: 7(66.4880275726 ms)/0.00153195462042)
node: ('grip0', 'floor', 0)->('goal', 'floor', 0) = success
node: ('goal', 'grip0', 0)->('goal', 'goal', 0) = fail
node: ('goal', 'floor', 0)->('goal', 'grip0', 0) = fail
branching: 3->21 (0.81/300.0 s, steps/err: 18(33.035993576 ms)/0.00127981043251)
node: ('floor', 'floor', 0)->('grip0', 'floor', 0) = fail
node: ('goal', 'grip0', 0)->('goal', 'goal', 0) = fail
node: ('grip0', 'floor', 1)->('floor', 'floor', 1) = success
branching: 4->22 (0.85/300.0 s, steps/err: 35(102.718114853 ms)/0.00187579805428)
node: ('floor', 'floor', 1)->('floor', 'grip0', 1) = fail
node: ('goal', 'grip0', 0)->('goal', 'go

node: ('grip0', 'floor', 2)->('floor', 'floor', 2) = success
node: ('grip0', 'floor', 1)->('goal', 'floor', 1) = success
branching: 37->41 (1.46/300.0 s, steps/err: 15(33.1192016602 ms)/0.00208197806597)
branching: 20->40 (1.44/300.0 s, steps/err: 33(45.5830097198 ms)/0.00130349617431)
node: ('grip0', 'floor', 1)->('grip0', 'floor', 2) = fail
node: ('grip0', 'floor', 2)->('goal', 'floor', 2) = success
node: ('goal', 'grip0', 0)->('goal', 'floor', 0) = fail
node: ('grip0', 'floor', 1)->('grip0', 'floor', 2) = fail
branching: 37->42 (1.48/300.0 s, steps/err: 25(55.5629730225 ms)/0.00112634953662)
node: ('floor', 'floor', 0)->('floor', 'grip0', 0) = fail
node: ('goal', 'floor', 1)->('grip0', 'floor', 1) = fail
node: ('floor', 'floor', 1)->('grip0', 'floor', 1) = fail
node: ('goal', 'floor', 0)->('grip0', 'floor', 0) = fail
node: ('floor', 'grip0', 0)->('floor', 'goal', 0) = success
node: ('floor', 'floor', 0)->('floor', 'floor', 1) = fail
branching: 31->43 (1.55/300.0 s, steps/err: 42(111

node: ('floor', 'grip0', 0)->('floor', 'goal', 0) = fail
node: ('grip0', 'floor', 1)->('goal', 'floor', 1) = success
node: ('goal', 'floor', 0)->('goal', 'grip0', 0) = success
node: ('floor', 'floor', 1)->('floor', 'grip0', 1) = fail
branching: 11->60 (1.96/300.0 s, steps/err: 35(103.843212128 ms)/0.00202402461321)
node: ('floor', 'floor', 2)->('floor', 'floor', 3) = fail
node: ('goal', 'floor', 2)->('grip0', 'floor', 2) = fail
node: ('floor', 'floor', 2)->('grip0', 'floor', 2) = fail
node: ('goal', 'grip0', 0)->('goal', 'grip0', 1) = fail
branching: 20->59 (1.97/300.0 s, steps/err: 40(99.1342067719 ms)/0.00148269086237)
node: ('floor', 'floor', 0)->('floor', 'grip0', 0) = fail
node: ('goal', 'grip0', 0)->('goal', 'goal', 0) = fail
node: ('floor', 'floor', 0)->('floor', 'floor', 1) = fail
node: ('goal', 'grip0', 2)->('goal', 'grip0', 3) = fail
node: ('floor', 'grip0', 0)->('floor', 'goal', 0) = success
node: ('grip0', 'floor', 1)->('goal', 'floor', 1) = success
node: ('floor', 'floor',

branching: 1->77 (2.35/300.0 s, steps/err: 21(137.313842773 ms)/0.00185962575394)
branching: 31->76 (2.35/300.0 s, steps/err: 32(57.3871135712 ms)/0.00123042733191)
node: ('goal', 'floor', 2)->('goal', 'grip0', 2) = success
node: ('grip0', 'goal', 0)->('goal', 'goal', 0) = fail
node: ('goal', 'grip0', 1)->('goal', 'goal', 1) = fail
node: ('floor', 'floor', 1)->('floor', 'floor', 2) = fail
branching: 47->78 (2.39/300.0 s, steps/err: 23(102.797031403 ms)/0.00117806902148)
node: ('goal', 'floor', 1)->('goal', 'floor', 2) = fail
node: ('floor', 'grip0', 0)->('floor', 'grip0', 1) = fail
node: ('floor', 'goal', 0)->('floor', 'grip0', 0) = success
node: ('floor', 'goal', 0)->('floor', 'goal', 1) = fail
node: ('goal', 'grip0', 1)->('goal', 'goal', 1) = fail
node: ('floor', 'goal', 0)->('floor', 'goal', 1) = fail
node: ('goal', 'floor', 0)->('grip0', 'floor', 0) = success
branching: 55->79 (2.44/300.0 s, steps/err: 74(286.550998688 ms)/0.0013026879043)
branching: 15->80 (2.44/300.0 s, steps/err

...

node: ('floor', 'goal', 4)->('grip0', 'goal', 4) = fail
node: ('floor', 'floor', 13)->('grip0', 'floor', 13) = success
node: ('goal', 'grip0', 9)->('goal', 'grip0', 10) = fail
branching: 2120->2391 (94.23/300.0 s, steps/err: 4(264.858961105 ms)/0.00145324347327)
node: ('grip0', 'floor', 8)->('goal', 'floor', 8) = success
node: ('grip0', 'floor', 12)->('grip0', 'floor', 13) = fail
branching: 1241->2392 (94.25/300.0 s, steps/err: 29(283.457994461 ms)/0.00134296157416)
node: ('goal', 'grip0', 5)->('goal', 'goal', 5) = fail
node: ('goal', 'floor', 5)->('grip0', 'floor', 5) = success
node: ('floor', 'floor', 2)->('grip0', 'floor', 2) = fail
node: ('goal', 'grip0', 5)->('goal', 'grip0', 6) = fail
node: ('grip0', 'goal', 10)->('goal', 'goal', 10) = fail
branching: 1329->2393 (94.29/300.0 s, steps/err: 36(51.5699386597 ms)/0.00183897478042)
node: ('goal', 'grip0', 13)->('goal', 'grip0', 14) = fail
node: ('grip0', 'goal', 8)->('floor', 'goal', 8) = success
node: ('goal', 'floor', 9)->('goal', '

node: ('floor', 'grip0', 9)->('floor', 'goal', 9) = success
node: ('floor', 'floor', 2)->('grip0', 'floor', 2) = fail
branching: 641->2409 (94.97/300.0 s, steps/err: 23(138.33117485 ms)/0.000645101186231)
node: ('floor', 'grip0', 15)->('floor', 'floor', 15) = fail
node: ('floor', 'floor', 14)->('floor', 'floor', 15) = fail
node: ('goal', 'grip0', 12)->('goal', 'goal', 12) = fail
node: ('floor', 'grip0', 11)->('floor', 'floor', 11) = fail
node: ('floor', 'grip0', 2)->('floor', 'floor', 2) = fail
node: ('goal', 'grip0', 3)->('goal', 'grip0', 4) = fail
node: ('grip0', 'goal', 3)->('grip0', 'goal', 4) = fail
node: ('goal', 'floor', 7)->('grip0', 'floor', 7) = fail
node: ('floor', 'floor', 0)->('floor', 'floor', 1) = fail
node: ('grip0', 'goal', 4)->('grip0', 'goal', 5) = fail
node: ('grip0', 'floor', 3)->('grip0', 'floor', 4) = fail
node: ('grip0', 'goal', 4)->('goal', 'goal', 4) = fail
node: ('goal', 'grip0', 13)->('goal', 'floor', 13) = success
branching: 2074->2410 (95.3/300.0 s, steps/

branching: 1950->2427 (95.82/300.0 s, steps/err: 2(43.1618690491 ms)/0.00191164483914)
node: ('grip0', 'goal', 14)->('grip0', 'goal', 15) = fail
node: ('goal', 'grip0', 7)->('goal', 'floor', 7) = fail
node: ('goal', 'grip0', 10)->('goal', 'goal', 10) = fail
node: ('floor', 'grip0', 6)->('floor', 'goal', 6) = success
node: ('floor', 'floor', 11)->('floor', 'grip0', 11) = fail
node: ('grip0', 'goal', 14)->('grip0', 'goal', 15) = fail
node: ('grip0', 'goal', 5)->('goal', 'goal', 5) = fail
node: ('floor', 'grip0', 2)->('floor', 'grip0', 3) = fail
branching: 473->2428 (95.87/300.0 s, steps/err: 60(241.397857666 ms)/0.00149124689608)
node: ('floor', 'grip0', 5)->('floor', 'grip0', 6) = fail
node: ('floor', 'floor', 10)->('floor', 'grip0', 10) = success
branching: 1581->2429 (95.9/300.0 s, steps/err: 38(201.900005341 ms)/0.00222460905212)
node: ('floor', 'grip0', 0)->('floor', 'floor', 0) = fail
node: ('goal', 'floor', 14)->('goal', 'grip0', 14) = fail
node: ('goal', 'grip0', 1)->('goal', 'gr

node: ('grip0', 'goal', 7)->('grip0', 'goal', 8) = success
branching: 1815->2445 (96.78/300.0 s, steps/err: 59(755.53894043 ms)/0.00214376702806)
node: ('floor', 'floor', 4)->('floor', 'floor', 5) = fail
node: ('grip0', 'floor', 1)->('floor', 'floor', 1) = success
branching: 20->2446 (96.82/300.0 s, steps/err: 25(516.071081161 ms)/0.00158407830474)
node: ('grip0', 'floor', 0)->('goal', 'floor', 0) = success
node: ('grip0', 'goal', 2)->('grip0', 'goal', 3) = fail
branching: 210->2447 (96.82/300.0 s, steps/err: 61(956.640958786 ms)/0.00195771666761)
node: ('grip0', 'floor', 13)->('grip0', 'floor', 14) = fail
node: ('goal', 'floor', 6)->('grip0', 'floor', 6) = success
node: ('grip0', 'goal', 0)->('grip0', 'goal', 1) = success
branching: 1836->2448 (96.86/300.0 s, steps/err: 34(70.5909729004 ms)/0.00147124568713)
node: ('goal', 'grip0', 1)->('goal', 'goal', 1) = fail
node: ('grip0', 'goal', 2)->('goal', 'goal', 2) = fail
branching: 1845->2449 (96.86/300.0 s, steps/err: 33(95.0810909271 ms)

======================= terminated 9: first answer acquired from other agent ===============================
branching: 1072->2460 (97.2/300.0 s, steps/err: 7(470.278978348 ms)/0.00160175078969)
======================= terminated 0: first answer acquired from other agent ===============================
node: ('floor', 'goal', 11)->('floor', 'grip0', 11) = success
branching: 1088->2462 (97.23/300.0 s, steps/err: 54(257.113933563 ms)/0.00187309221302)
======================= terminated 1: first answer acquired from other agent ===============================
======================= terminated 7: first answer acquired from other agent ===============================
node: ('grip0', 'goal', 4)->('floor', 'goal', 4) = success
branching: 424->2463 (97.31/300.0 s, steps/err: 53(251.981019974 ms)/0.000916268586284)
node: ('grip0', 'goal', 11)->('grip0', 'goal', 12) = fail
======================= terminated 15: first answer acquired from other agent ===============================
=============

In [23]:
print(gtimer)

plan: 	97220.0 ms/1 = 97219.633 ms (97219.633/97219.633)



## play searched plan

In [24]:
ppline.play_schedule(snode_schedule[:2], period=0.01)
pscene.set_object_state(initial_state)
gscene.show_pose(crob.home_pose)
time.sleep(0.5)
gscene.show_pose(crob.home_pose)

In [25]:
snode_schedule = ppline.add_return_motion(snode_schedule)

In [26]:
ppline.play_schedule(snode_schedule, period=0.03)